<a href="https://colab.research.google.com/github/DanialPahlavan/RAG/blob/main/atlas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Prepare Files

### 0.1 Clone project

In [1]:
!git clone https://github.com/facebookresearch/atlas

Cloning into 'atlas'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 144 (delta 35), reused 13 (delta 10), pack-reused 89 (from 1)
Receiving objects: 100% (144/144), 162.21 KiB | 13.52 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [2]:
cd atlas

/content/atlas


### 0.2 Install Pre Requirments

may needs restart runtime for prepare old libs

In [3]:
!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 GB 785.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 59.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.3.1+cu121
    Uninstalling torch-2.3.1+cu121:
      Successfully uninstalled torch-2.3.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.18.1+cu121
    Uninstalling torchvision-0.18.1+cu121:
      Successfully uninstalled torchvision-0.18.1+cu121
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.3.1+cu121
    Uninstalling torchaudio-2.3.1+cu121:
      Successfully uninstalled torchaudio-2.3.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the fol

In [4]:
import torch
print(torch.cuda.is_available())  # Should return True if the GPU is being used


True


In [5]:
!pip install faiss-gpu==1.7.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.3 MB/s eta 0:00:00


In [6]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.2/248.2 kB 15.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 62.3 MB/s eta 0:00:00
  Created wheel for fairscale: filename=fairscale-0.4.6-py3-none-any.whl size=307221 sha256=53c8f8d9a2e1b324aec3bf1dcabd5bb3aacfce578ead2fa7124193bf8627debd
  Stored in directory: /root/.cache/pip/wheels/a1/58/3d/e114952ab4a8f31eb9dae230658450afff986b211a5b1f2256
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=fccacfedcc259ecb31fe202c9b6c867135a25428

## 1. Download preprocess and datasets

### 1.1 download Corpora



| Corpus Name                               | Corpus Download Key                                 | Description                                                                    | Size |
|--------------------------------------------|----------------------------------------------------|--------------------------------------------------------------------------------|-------|
| enwiki-dec2017                               | `corpora/wiki/enwiki-dec2017`                        | Wikipedia dump from Dec 2017, preprocessed into passages                         | 30.4M (26.9M text, 2.7M infobox) |
| enwiki-dec2018                               | `corpora/wiki/enwiki-dec2018`                        | Wikipedia dump from Dec 2018, preprocessed into passages (recommended for NQ, TriviaQA) | 32.1M (28.4M text, 3.7M infobox) |
| enwiki-aug2019                               | `corpora/wiki/enwiki-aug2019`                        | Wikipedia dump from August 2019, preprocessed into passages                         | 33.1M (29.4M text, 3.8M infobox) |
| enwiki-dec2020                               | `corpora/wiki/enwiki-dec2020`                        | Wikipedia dump from Dec 2020, preprocessed into passages                         | 35.6M (31.5M text, 4.1M infobox) |
| enwiki-dec2021                               | `corpora/wiki/enwiki-dec2021`                        | Wikipedia dump from Dec 2021, preprocessed into passages                         | 37.5M (33.1M text, 4.3M infobox) |


aware: ""We cannot open-source the common-crawl indices used in the paper at this time."" source : github repo of this project



In [7]:
mkdir data

In [8]:
#The corpus that was used to train the model can be downloaded with :
#The $DATA_DIR folder is used to store everything we download.

#!python preprocessing/download_corpus.py --corpus {corpus download key} --output_directory ${DATA_DIR}
#if you assigned DAT_DIR use below:
#!python preprocessing/download_corpus.py --corpus corpora/wiki/enwiki-dec2018 --output_directory $DATA_DIR
!python preprocessing/download_corpus.py --corpus corpora/wiki/enwiki-dec2018 --output_directory /content/atlas/data

### 1.2 models

We are open-sourcing pretrained Atlas models at base, large, xl and xxl sizes. These include both the pretrained retriever and reader weights. In addition, we're open-sourcing our strongest-performing fully-finetuned NaturalQuestions Atlas models, for users who want to perform state-of-the-art QA inference (or finetune them on other QA tasks). Models can be downloaded as follows:


| Model | Model Download Key | Description | Parameters (reader / retriever) |
|---|---|---|---|
| Atlas-xxl | `models/atlas/xxl` | Pretrained Atlas XXL model | 11B / 110M |
| Atlas-xl | `models/atlas/xl` | Pretrained Atlas XL model | 3B / 110M |
| Atlas-large | `models/atlas/large` | Pretrained Atlas large model | 770M / 110M |
| Atlas-base | `models/atlas/base` | Pretrained Atlas base model | 220M / 110M |
| NQ-finetuned Atlas-xxl | `models/atlas_nq/xxl` | Atlas XXL model, finetuned on Natural Question | 11B / 110M |
| NQ-finetuned Atlas-xl | `models/atlas_nq/xl` | Atlas XL model, finetuned on Natural Question | 3B / 110M |
| NQ-finetuned Atlas-large | `models/atlas_nq/large` | Atlas large model, finetuned on Natural Question | 770M / 110M |
| NQ-finetuned Atlas-base | `models/atlas_nq/base` | Atlas base model, finetuned on Natural Question | 220M / 110M |



In [9]:
# To download the model checkpoints:

#python preprocessing/download_model.py --model {model download key} --output_directory ${DATA_DIR}

!python preprocessing/download_model.py --model models/atlas/xl --output_directory /content/atlas/data

### 1.3 Pre-built Indices
Atlas will automatically build an index if none is provided. This is convenient, but can take a long time, especially with fewer GPU workers, or if the index is very large.

We have therefore made precomputed indices available for download for the wiki-dec2018 corpus for the pretrained Atlas checkpoints, and for nq-finetuned Atlas checkpoints

These can be downloaded as follows :


| Index | Index Download Key | Corresponding Model | Description |
|---|---|---|---|
| Atlas XXL wiki-dec2018 index | `indices/atlas/wiki/xxl` | `models/atlas/xxl` | Precomputed index for the wiki-dec2018 corpus for the pretrained Atlas-xxl model |
| Atlas XL wiki-dec2018 index | `indices/atlas/wiki/xl` | `models/atlas/xl` | Precomputed index for the wiki-dec2018 corpus for the pretrained Atlas-xl model |
| Atlas large wiki-dec2018 index | `indices/atlas/wiki/large` | `models/atlas/large` | Precomputed index for the wiki-dec2018 corpus for the pretrained Atlas-large model |
| Atlas base wiki-dec2018 index | `indices/atlas/wiki/base` | `models/atlas/base` | Precomputed index for the wiki-dec2018 corpus for the pretrained Atlas-base model |
| Atlas-nq XXL wiki-dec2018 index | `indices/atlas_nq/wiki/xxl` | `models/atlas_nq/xxl` | Precomputed index for the wiki-dec2018 corpus for the natural-questions-finetuned Atlas xxl model |
| Atlas-nq XL wiki-dec2018 index | `indices/atlas_nq/wiki/xl` | `models/atlas/xl` | Precomputed index for the wiki-dec2018 corpus for the natural-questions-finetuned Atlas xl model |
| Atlas-nq large wiki-dec2018 index | `indices/atlas_nq/wiki/large` | `models/atlas/large` | Precomputed index for the wiki-dec2018 corpus for the natural-questions-finetuned Atlas large model |
| Atlas-nq base wiki-dec2018 index | `indices/atlas_nq/wiki/base` | `models/atlas/base` | Precomputed index for the wiki-dec2018 corpus for the natural-questions-finetuned Atlas base model |



In [ ]:
# To download the prebuilt indexes:

#python preprocessing/download_index.py --index {index download key} --output_directory ${DATA_DIR}
!python preprocessing/download_index.py --index indices/atlas/wiki/xl --output_directory /content/atlas/data

In [ ]:
# Finally, to download the NQ data:

!python preprocessing/prepare_qa.py --output_directory /content/atlas/data

## 3. Finetuning - [needs a machine with 8 GPUs (A100 with 80 GB of RAM) ]



### 3.1 make own datasets

make question and asnwer with this pattern and save jsonl file in this place

$DATA_DIR/nq_data/train.100-shot.jsonl

pattern:

**{"question": "when is season 2 of punisher coming out on netflix", "answers": ["in 2019"]}**



we create a folder to collect the experiments output and the run logs:

In [ ]:
!mkdir $DATA_DIR/experiments

### 3.2 finetune model
We now pass all the paths where the models and data were downloaded to the torchrun command. In a machine with 8 GPUs (A100 with 80 GB of RAM), we can run the following to conduct the finetuning:



In [ ]:
torchrun --standalone --nnodes 1 --nproc_per_node 8  finetune_qa.py --train_data
$DATA_DIR/nq_data/train.100-shot.jsonl --eval_data $DATA_DIR/nq_data/test.jsonl --name
"my_finetuning_experiment_baseline_xl"  --checkpoint_dir $DATA_DIR/experiments/ --total_steps 31
--index_mode flat --model_path $DATA_DIR/models/atlas/xl --load_index_path
$DATA_DIR/indices/atlas/wiki/xl --reader_model_type  google/t5-xl-lm-adapt

### 3.3 finetune model - alternative
Alternatively, we can conduct the finetuning using a product quantized index, with light compression corresponding to a code size of 192. In order to use a Faiss product-quantized index, we select --index_mode faiss and --faiss_index_type pq respectively and compression parameter --faiss_code_size 192:.



In [ ]:
torchrun --standalone --nnodes 1 --nproc_per_node 8  finetune_qa.py --train_data
$DATA_DIR/nq_data/train.100-shot.jsonl --eval_data $DATA_DIR/nq_data/test.jsonl --name
"my_finetuning_experiment_pq_192_xl"  --checkpoint_dir $DATA_DIR/experiments/ --total_steps 31
--index_mode faiss --faiss_index_type  pq --faiss_code_size 192 --model_path
$DATA_DIR/models/atlas/xl --load_index_path $DATA_DIR/indices/atlas/wiki/xl --reader_model_type
google/t5-xl-lm-adapt

### 3.4 finetune model - alternative

We can also use a more aggressive compression parameter — corresponding to a code size of 64 — and conduct the finetuning with the product quantized index using only 2 GPUs:



In [ ]:
torchrun --standalone --nnodes 1 --nproc_per_node 2  finetune_qa.py --train_data
$DATA_DIR/nq_data/train.100-shot.jsonl --eval_data $DATA_DIR/nq_data/test.jsonl --name
"my_finetuning_experiment_pq_64_xl"  --checkpoint_dir $DATA_DIR/experiments/ --total_steps 31
--index_mode faiss --faiss_index_type  pq --faiss_code_size 64 --model_path $DATA_DIR/models/atlas/xl
--load_index_path $DATA_DIR/indices/atlas/wiki/xl --reader_model_type  google/t5-xl-lm-adapt

## others - Dump info -need clean it


### Tasks


Atlas can train (or evaluate) on any supervised learning task which can be formulated in a "seq2seq" format, where there is a sequence of 1 or more tokens comprising an input query and a sequence of 1 or more tokens comprising an output target. For example, a query might be a question, Where is the Bermuda Triangle?, and a target might be the answer to that question, Western part of the North Atlantic Ocean. This way of modelling will be familiar to users of models like T5 or BART. Anywhere these models could be used, Atlas can be used too, using the exact same data: Atlas will learn to retrieve passages from its retrieval index by itself - annotations for associating passages to (query, target) pairs are not used.

The Atlas codebase configures what task it is doing, and what evaluation metrics to call using the --task command line argument. We have implemented a base task, with only the most basic support for seq2seq training, but provide more fully-featured functionality for Masked Language Modelling (mlm), Language Modelling (lm), Wikipedia section generation (section), Open-domain QA (QA), Multiple choice QA (multiple_choice), fact checking (fever), and the KILT suite (kilt), All tasks expect input data formatted as jsonl format, but the specific field names are task specific. Some tasks have additional command line args, and specialized evaluation. Adding new tasks is straightforward, and described here.

The tasks are described in more detail below, and most have example commands in examples/{task}/ .

Base Task
Masked Language Modelling
Language Modelling
Wikipedia Section Generation
Open-Domain Question Answering (e.g. NaturalQuestions, TriviaQA, TempLama)
Multiple Choice Question Answering (e.g. MMLU)
FEVER Fact Verification
KILT

### 2.1 Base Task

This is the most basic task available, and is probably not the best option for you, especially if your task closely resembles one the other implemented tasks.

Specify this task by passing --task base to either train.py or evaluate.py

Train/validation/test data for this task should consist of jsonl files, which should be passed to train.py or evaluate.py space-separated lists to --train_data train_file_1.jsonl train_file_2.jsonl, and --eval_data eval_file_1.jsonl eval_file_2.jsonl etc. This task expects input files to have a query field with the input query string and a target field with the output query string, e.g.:

{"query": "input to Atlas", "target": "desired generation from Atlas"}

The evaluation loop will calculate evaluation loss and the fraction of eval data examples where Atlas generates an output that exactly matches the target. If you pass --write_results to the script, Atlas predictions on the eval data will be written to the save checkpoint directory with the following format:


{"query": "input to Atlas", "answers": ["desired generation from Atlas"], "generation": "Atlas's prediction for the query", "passages": ["list of retrieved passages"]}

### 2.2 Masked Language Modelling
The Masked Language modelling task implements the Masked Language Modelling pretraining task as introduced by T5. This is the task we use to pretrain the main Atlas in the paper.

Specify this task by passing --task mlm to train.py.

Train/validation/test data for this task should consist of jsonl files, which should be passed to train.py as --train_data train_file_1.jsonl train_file_2.jsonl, and --eval_data eval_file_1.jsonl eval_file_2.jsonl etc. These files should be comprised of JSON objects with the following format:

{
  "text": "text passage to apply noise to and train to de-noise",
  "id": "unique id of text passage"
  ... # you can have other fields you want to keep around for ease of analysis, but they wont actually be used
}


The intention is that the same files that you use for the retrieval corpus, (passed to --passages) can be used as training data. The task will apply the T5 noise function to text field, to automatically create inputs and target generations.

The MLM task will prevent Atlas from retrieving the passage that it is trying to de-noise. It does this by filtering out any passage from retrieved results which have same id field as the instance Atlas is de-noising. This functionality is important if the de-noising training data and the passages Atlas is retrieving from are the same corpus.

This task has the following task specific args:

  --mlm_noise_density MLM_NOISE_DENSITY
      how much of an input text should be masked by masking spans (default: 0.15)
  --mlm_mean_noise_span_length MLM_MEAN_NOISE_SPAN_LENGTH
      average length of an MLM masking span (default: 3)
  --min_words_per_lm_instance MIN_WORDS_PER_LM_INSTANCE
      Instances with fewer than min_words_per_lm_instance instances will be skipped for MLM/LM/Section generation (default: None)

If you pass --write_results, Atlas will write its mask-filling predictions to file.

Atlas will log the following evaluation metrics for MLM during its evaluation loop:

eval_loss: evaluation reader loss of generated mlm mask-fill spans
accuracy: fraction of perfectly de-noised mask-fill spans
f1: token f1 fraction of correct de-noised mask-fill spans
rouge_1: rouge 1 score of generated mask-fill spans relative to the gold reference masked spans
rouge_2: rouge 2 score of generated mask-fill spans relative to the gold reference masked spans
rouge_L: rouge L score of generated mask-fill spans relative to the gold reference masked spans



### Language Modelling

Atlas can be trained to do Left-to-Right Language Modeling by passing --task lm to train.py.

Train/validation/test data for this task should consist of jsonl files, which should be passed to train.py as --train_data train_file_1.jsonl train_file_2.jsonl, and --eval_data eval_file_1.jsonl eval_file_2.jsonl etc. These files should be comprised of JSON objects with the following format:

{
  "text": "text passage to train Atlas to generate",
  "id": "unique id of text passage"
  ... # you can have other fields you want to keep around for ease of analysis, but they wont actually be used
}

The intention is that the same files that you use for the retrieval corpus, (passed to --passages) can be used as training data. The task will preprocess the text field automatically, dividing it into two random segments - the left part serves as conditioning context, and the right part is the text the Atlas model will be trained to generate as a continuation.

The LM task will prevent Atlas from retrieving the same passage that it is trying to generate. It does this by filtering out any passage from retrieved results which have same id field as the instance Atlas is generating. This functionality is important if the de-noising training data and the passages Atlas is retrieving from are the same corpus.

This task has the following task specific args:

  --min_words_per_lm_instance MIN_WORDS_PER_LM_INSTANCE
      Instances with fewer than min_words_per_lm_instance instances will be skipped for  MLM/LM/Section generation (default: None)
  --min_lm_context_ratio MIN_LM_CONTEXT_RATIO
      Splits text into two segments for language modelling.' 'Left segment is conditioning context, right segment is for generating.' 'The left segment must be more than min_lm_context_ratio of
      the right segment (default: 0.5)
  --max_lm_context_ratio MAX_LM_CONTEXT_RATIO
      Splits text into two segments for language modelling.' 'Left segment is conditioning context, right segment is for generating.' 'The left segment must be less than max_lm_context_ratio
      of the right segment (default: 0.5)

If you pass --write_results, Atlas will write its lm predictions to file.

Atlas will log the following evaluation metrics for LM during its evaluation loop:

eval_loss: evaluation reader loss of continuations for the reference data
accuracy: fraction of perfectly predicted continuations
f1: token f1 fraction of correct generated continuations
rouge_1: rouge 1 score of generated continuations relative to the gold reference continuations
rouge_2: rouge 2 score of generated continuations relative to the gold reference continuations
rouge_L: rouge L score of generated continuations relative to the gold reference continuations

### Wikipedia Section Generation

Atlas can be trained to generate the text of a wikipedia passage given its title and section title, by passing --task section to train.py.

Train/validation/test data for this task should consist of jsonl files, which should have the form of the text-list-100-sec.jsonl files in the wikipedia dumps. These can be obtained by following the instructions in Available Data and Models for download, for example the training file: enwiki-dec2018/text-list-100-sec.jsonl. These files should be comprised of JSON objects, one per line, with the following format:

{
  "id": "3793043",
  "title": "Bermuda Triangle",
  "section": "Compass variations",
  "text": " Compass problems are one of the cited phrases in many Triangle incidents. While some have theorized that unusual local magnetic anomalies may exist in the area, such anomalies have not been found. Compasses have natural magnetic variations in relation to the magnetic poles, a fact which navigators have known for centuries."
}


The task will automatically format the input query to the model as "{Title}, {Section}" - e.g. in this example, the input to Atlas will be constructed as Bermuda Triangle, Compass Variations. The output will be the text field of the example. The section task will prevent Atlas from retrieving the same passage that it is trying to generate. It does this by filtering out any passage from retrieved results which have same id field as the instance Atlas is generating.

This task has the following task specific args:

  --min_words_per_lm_instance MIN_WORDS_PER_LM_INSTANCE
      Instances with fewer than min_words_per_lm_instance instances will be skipped for MLM/LM/Section generation (default: None)

If you pass --write_results, Atlas will write its generated predictions for the text for Wikipedia sections to file.

Atlas will log the following evaluation metrics for section during its evaluation loop:

eval_loss: evaluation reader loss of continuations for the reference data
accuracy: fraction of perfectly predicted continuations
f1: token f1 fraction of correct generated continuations
rouge_1: rouge 1 score of generated continuations relative to the gold reference continuations
rouge_2: rouge 2 score of generated continuations relative to the gold reference continuations
rouge_L: rouge L score of generated continuations relative to the gold reference continuations

### Open-Domain Question Answering (e.g. NaturalQuestions, TriviaQA, TempLama)

Atlas can be trained to answer open-domain QA questions by passing --task qa to train.py or evaluate.py. There is a worked example of QA in the Getting Started and Codebase at a Glance section. We use this task for the NaturalQuestions, TriviaQA and TempLama datasets in the paper.

Train/validation/test data for this task should consist of jsonl files, which should be passed to train.py as --train_data train_file_1.jsonl train_file_2.jsonl, and --eval_data eval_file_1.jsonl eval_file_2.jsonl etc. Files should have one JSON instance per line with the following format:

{
  "question": "where is the bermuda triangle",
  "answers": ["Western part of the North Atlantic Ocean"],
   ... # you can have other fields you want to keep around for ease of analysis, but they wont actually be used
}

The question will be formatted according to the task specific argument --qa_prompt_format, which defaults to question: {question} answer: <extra_id_0>. For example above, the question would be automatically formatted as input queries to Atlas as question: where is the bermuda triangle answer: <extra_id_0>. The supervision target is obtained from the target field. If this field does not exist, the supervision target will get selected at random from the available answers in the answers field, and formatted as <extra_id_0> {answer}.

If you pass --write_results, Atlas will write its predicted answers to file.

Atlas will log the following evaluation metrics for open domain QA during its evaluation loop:

eval_loss: evaluation reader loss of evaluation answers.
exact_match: Open-domain QA exact match score of generated answers
f1: Open-domain QA F1 score of generated answers
Natural Questions & TriviaQA
You can download the NaturalQuestions and TriviaQA data by calling:

python preprocessing/prepare_qa.py --output_directory ${DATA_DIR}

which will download train.jsonl, train.64-shot.jsonl (the fewshot training dataset we use), dev.jsonl and test.jsonl to ${DATA_DIR}/data/nq_data and ${DATA_DIR}/data/triviaqa_data.

Example scripts for running fewshot and standard finetuning and evaluation with a wikipedia index for NQ can be found in examples/nq. This script can be used for TriviaQA by swapping the train/dev/test files.

TempLama
We defined a cloze-question answering task for assessing index faithfulness and temporal transfer, derived from the TempLAMA dataset.

You can download the TempLAMA data and create and format our derived dataset by calling the following script:

python preprocessing/prepare_templama.py --output_directory ${DATA_DIR}

which will create the files temp_lama.train.2017.jsonl, temp_lama.valid.2017.jsonl, temp_lama.test.2017.jsonl, temp_lama.train.2020.jsonl, temp_lama.valid.2020.jsonl, temp_lama.test.2020.jsonl under ${DATA_DIR}/data/templama_data/. These files will contain cloze questions, with answers specific to that year.

Example scripts for running training and evaluation for TempLama can be found at examples/templama. (note the use of qa_prompt_format {question}, which switches off the automatic QA prompt formatting used for TriviaQA and NQ)


### Multiple Choice Question Answering (e.g. MMLU)

Atlas can be trained to answer multiple choice questions by passing --task multiple_choice to train.py or evaluate.py. We use this task for our experiments with MMLU.

Train/validation/test data for this task should consist of jsonl files, which should be passed to train.py as --train_data train_file_1.jsonl train_file_2.jsonl, and --eval_data eval_file_1.jsonl eval_file_2.jsonl etc. Files should have one JSON instance per line with the following format:

{
  "question": "Which of the following is the body cavity that contains the pituitary gland?",
  "options": {
    "A": "Abdominal",
    "B": "Cranial",
    "C": "Pleural",
    "D": "Spinal"
    ... # you can have more (or fewer) answer options as long as they have alphabetically consecutive upper case letter keys, starting at A
  },
  "answer": "B",
  ... # you can have other fields you want to keep around for ease of analysis, but they wont actually be used
}

These will get automatically formatted into input queries for Atlas of the form question: {question} answers: (A) {options['A']} (B) {options['B']} (C) {options['C']} (D) {options['D']} Answer: <extra_id_0>, with target generations of the format <extra_id_0> {answer letter}. The example above would get formatted to: question: {Which of the following is the body cavity that contains the pituitary gland? answers: (A) Abdominal (B) Cranial (C) Pleural (D) Spinal Answer: <extra_id_0>, with the target generation {extra_id_0} B.

Multiple-Choice QA has the following task specific args:

  --multiple_choice_num_options
      How many choice options for multiple choice QA (MMLU is 4) (default: 4)
  --multiple_choice_train_permutations {single,cyclic,all}
      Whether to train with answer order permutations When training on multiple choice (e.g. MMLU). Can improve results by de-biasing models's preferences for arbitrary answer orderings. Recommend
      training with 'all'. single: no permutations. cyclic: cyclic permutations. all: all possible answer order permutations' (default: single)
  --multiple_choice_eval_permutations {single,cyclic,all}
      Whether to evaluate with answer order permutations for multiple choice (e.g. MMLU). Can improve results by de-biasing models's preferences for arbitrary answer orderings. Best results with
      'all' but very slow. 'cyclic' is a good compromise. single: no permutations. cyclic: cyclic permutations. all: all possible answer order permutations' (default: single)

The permutation options will automatically duplicate the inputs, but with the answer orders permuted (e.g. With "A" now being "cranial", "B" being "pleural" etc.) This improves results for when we have very small amounts of supervised data (or zeroshot). The code will automatically marginalize across results for evaluation permutations for you, in the case you use --multiple_choice_eval_permutations option cyclic or all. More details on the permutation de-biasing can be found in the appendix of Atlas: Few-shot Learning with Retrieval Augmented Language Models.

If you pass --write_results, Atlas will write its predicted answers to file, with the following format:

{
  "question": "the prompt-template applied input",
  "generation": "answer letter choice with highest probability after marginalizing across permutations",
  "choice_probs": "the probability of each answer choice (normalized over total answer options)",
  "all_probs": "the un-marginalized answer probabilities from all the answer order permutations",
  "permutations": ["the list of prediction objects for each permutation of the answer ordering"]
}

MMLU
A dedicated ReadMe is available for running MMLU experiments here. There is a tool to download and preprocess the MMLU data, and example scripts for running each of the experimental settings that we explore with MMLU are available examples/mmlu. These are documented in detail in the MMLU Dedicated Readme.



### FEVER Fact Verification

Atlas can be trained to classify textual claims as "SUPPORTED", "REFUTED" or "NOT_ENOUGH_INFO" by a corpus, such as for the FEVER task by using --task fever to train.py or evaluate.py.

You can download the FEVER data by calling the following script:

python preprocessing/prepare_fever.py --output_directory ${DATA_DIR}

Train/validation/test data for this task should consist of jsonl files, which should be passed to train.py as --train_data train_file_1.jsonl train_file_2.jsonl, and --eval_data eval_file_1.jsonl eval_file_2.jsonl etc. Files should have one JSON instance per line with the following format:

{
  "claim": "the claim to assess",
  "label": "either 'SUPPORTS', 'REFUTES' or 'NOT ENOUGH INFO'",
   ... # you can have other fields you want to keep around for ease of analysis, but they wont actually be used
}


Atlas will automatically process these instances, and format them for input as question: {claim} answer: <extra_id_0> and the output as <extra_id_0> {true, false or maybe}. If you pass --write_results, Atlas will write its predicted labels to file. Atlas will log the following evaluation metrics for open domain QA during its evaluation loop:

accuracy: how many claims were correctly classified by the model.

### KILT


Atlas can be trained to perform KILT tasks by using --task kilt to train.py or evaluate.py.

KILT data can be obtained from here

Train/validation/test data for this task should consist of jsonl files, which should be passed to train.py as --train_data train_file_1.jsonl train_file_2.jsonl, and --eval_data eval_file_1.jsonl eval_file_2.jsonl etc. Files should have one JSON instance per line with the following format (i.e. the codebase will accept the KILT format directly)

{'id': # original data point id if available otherwise unique id
 'input': # question / claim / sentence / etc
 'output': [ # each element might contain an answer, a provenance or both
    {
    'answer': # answer in textual form
    'provenance': [
        # evidence set for the answer from the KILT ks
        {
            'wikipedia_id':  # *mandatory*
            'title':
            'section':
            'start_paragraph_id':
            'start_character':
            'end_paragraph_id':
            'end_character':
            'bleu_score': # wrt original evidence
            'meta': # dataset/task specific
        }
        ]
      }
    ]
 'meta': # dataset/task specific
 }

Atlas will automatically process these instances appropriately, into Atlas] query inputs based on the input field and target generations based on the answer fields

If you pass --write_results, Atlas will write its predicted labels to file.

Atlas will log the following evaluation metrics for open domain QA during its evaluation loop:

accuracy: how often generations exactly match the reference
exact_match: how often generations exactly match the reference, with open-domain QA normalization applied
f1: the token level f1 score overlap between the generation and reference


